# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient("localhost:27017")

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
client.list_database_names()


['Ironhack', 'admin', 'config', 'local']

In [6]:
db = client.get_database("Ironhack")

In [7]:
db.list_collection_names()

['Restaurants', 'Offices']

In [8]:
o = db.get_collection("Offices")

In [9]:
o

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'Offices')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [9]:
Babelgum = list(o.find({'name':'Babelgum'}).limit(1))

In [10]:
#Babelgum

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [11]:
Big_companies = list(o.find({'number_of_employees':{"$gt": 5000}}).sort('number_of_employees',1).limit(20))

In [12]:
#Big_companies

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [13]:
Proy = {'name':1,'founded_year':1, "_id":0}
cond1 = {'founded_year':{'$gte': 2000}}
cond2 = {'founded_year':{'$lte': 2005}}

In [14]:
Companies_founded = list(o.find({"$and":[cond1,cond2]}, Proy).limit(5))

In [15]:
Companies_founded

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [17]:
Companies_valuation = list(o.find({'$and':[{'ipo.valuation_amount': {'$lte':100000000}},{'founded_year':{'$lte': 2000}}]}))

In [19]:
#Companies_valuation

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [20]:
Less_1000 = {'number_of_employees':{"$lte": 1000}}
founded_b = {'founded_year':{"$lte": 2005}}

In [23]:
List_1000_2005 = list(o.find({"$and":[Less_1000,founded_b]}).sort('number_of_employees',1).limit(10))
#List_1000_2005

### 6. All the companies that don't include the `partners` field.

In [24]:
No_partners = {"partners": {"$exists": False}}
No_partners

{'partners': {'$exists': False}}

### 7. All the companies that have a null type of value on the `category_code` field.

In [10]:
Null = list(o.find({'category_code': {'$type':"null"}},{"_id":0,"name":1}).limit(4))
Null

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [26]:
Proy_2 = {'name':1,'number_of_employees':1}
cond11 = {'number_of_employees':{'$gte': 100}}
cond22 = {'number_of_employees':{'$lte': 1000}}

In [28]:
Companies_100 = list(o.find({"$and":[cond11,cond22]}, Proy_2).limit(1))
Companies_100

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600}]

### 9. Order all the companies by their IPO price in a descending order.

In [34]:
Proy = {"ipo.valuation_amount":1,"_id":0}
Companies_ipo = list(o.find({"ipo.valuation_amount":1},Proy).sort('ipo.valuation_amount',-1))
Companies_ipo

[]

In [35]:
Companies_ipo = list(o.find({'ipo.valuation_amount':1}).sort('ipo.valuation_amount',-1))
Companies_ipo

[]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [37]:
List_10_c = list(o.find({'number_of_employees':1}).sort('number_of_employees',-1).limit(10))
#List_10_c[0:10]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [39]:
founded_2 = list(o.find({'founded_month':{'$gt':6}}).limit(1000))
#founded_2

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [41]:
Proy_3 = {'founded_year':1,'acquisition.price_amount':1}
Year_1 = {'founded_year':{'$lt':2000}}
Price_1 = {'acquisition.price_amount':{'$gt':1000000}}

In [47]:
More_10000 = list(o.find({"$and":[Year_1,Price_1]},Proy_3).limit(2))
More_10000

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297deb'),
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [48]:
Proy_4 = {'name':1,'acquisition':1}
Year_2 = {'acquisitions.acquired_year':{'$gt':2010}}
Price_2 = list(o.find({'acquisition.price_amount':{'$gt':1000000}}).sort('acquisition.price_amount',1))

In [50]:
#After_2010 = list(o.find({"$and":[Year_2,Price_2]},Proy_4))

#$or/$and/$nor entries need to be full objects, full error: {'ok': 0.0, 'errmsg': '$or/$and/$nor entries need to be full objects', 'code': 2, 'codeName': 'BadValue'}

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [54]:
Proy_5 = {'name':1,'founded_year':1}
By_founded = list(o.find({'number_of_employees':1},Proy_5).sort('founded_year',1).limit(5))
By_founded

[{'_id': ObjectId('52cdef7c4bab8bd675297f5c'),
  'name': 'Mixercast',
  'founded_year': None},
 {'_id': ObjectId('52cdef7d4bab8bd675298c57'),
  'name': 'LaterLoop',
  'founded_year': None},
 {'_id': ObjectId('52cdef7d4bab8bd675298988'),
  'name': 'GPokr',
  'founded_year': None},
 {'_id': ObjectId('52cdef7c4bab8bd6752981cb'),
  'name': 'Pixel Qi',
  'founded_year': None},
 {'_id': ObjectId('52cdef7c4bab8bd67529848e'),
  'name': 'Randomate',
  'founded_year': None}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [65]:
List_5 = list(o.find({'founded_day':{'$lt':8}}).sort('acquisition.price_amount',-1).limit(10))
#List_5

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [64]:
web = list(o.find({"category_code": {"$in":['web']}}).limit(5).sort('number_of_employees',1))
#web

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [61]:
Companies_EUR = list(o.find({'acquisition.price_amount':{'$gt':10000000}},{'acquisition.price_currency_code':'EUR'}).limit(2))
Companies_EUR

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'acquisition': {'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'acquisition': {'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [63]:
Proy_6 = {'name':1,'acquisition':1}
Companies_10 = list(o.find({'acquisition.acquired_month':{'$lt':4}},Proy_6).limit(10))
#Companies_10

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [170]:
proyect = {"name": 1, 'founded_year': 1, 'acquisition.acquired_year':1, '_id': 0}
cond1 = {'founded_year': {"$in": list(range(2000,2011))}}
cond2 = {'acquisition.acquired_year': {"$nin": list(range(0,2012))}}
list(o.find({"$and": [cond1,cond2]}, proyect).limit(3))

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code